## Lab assignment 02

### Neural Machine Translation in the wild
In the third homework you are supposed to get the best translation you can for the EN-RU translation task.

Basic approach using RNNs as encoder and decoder is implemented for you. 

Your ultimate task is to use the techniques we've covered, e.g.

* Optimization enhancements (e.g. learning rate decay)

* Transformer/CNN/<whatever you select> encoder (with or without positional encoding)

* attention/self-attention mechanism

* pretraining the language models (for decoder and encoder)

* or just fine-tunning BART/ELECTRA/... ;)

to improve the translation quality. 

__Please use at least three different approaches/models and compare them (translation quality/complexity/training and evaluation time).__

Write down some summary on your experiments and illustrate it with convergence plots/metrics and your thoughts. Just like you would approach a real problem.

In [1]:
# Thanks to YSDA NLP course team for the data
# (who thanks tilda and deephack teams for the data in their turn)

import os
path_do_data = 'data.txt'

if not os.path.exists(path_do_data):
    print("Dataset not found locally. Downloading from github.")
    !wget https://raw.githubusercontent.com/neychev/made_nlp_course/master/datasets/Machine_translation_EN_RU/data.txt -nc

Dataset not found locally. Downloading from github.
--2023-05-22 02:46:48--  https://raw.githubusercontent.com/neychev/made_nlp_course/master/datasets/Machine_translation_EN_RU/data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12905334 (12M) [text/plain]
Saving to: ‘data.txt’

data.txt            100%[===================>]  12.31M  --.-KB/s    in 0.1s    

2023-05-22 02:46:49 (111 MB/s) - ‘data.txt’ saved [12905334/12905334]



In [2]:
# Baseline solution BLEU score is quite low. Try to achieve at least __21__ BLEU on the test set. 
# The checkpoints are:

# * __21__ - minimal score to submit the homework, 30% of points

# * __25__ - good score, 70% of points

# * __27__ - excellent score, 100% of points

### Warning! The code below is deeeeeeeply deprecated and is is provided only as simple guide.
We suggest you to stick to most recent pipelines here, e.g. by Huggingface: 
* Example notebook: [link](https://github.com/huggingface/notebooks/blob/main/examples/translation.ipynb)
* Converting your own dataset to specific format: [link](https://discuss.huggingface.co/t/correct-way-to-create-a-dataset-from-a-csv-file/15686/15)

In [3]:
import numpy as np
import pandas as pd
import torch
import random
import matplotlib.pyplot as plt
import time

from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from transformers.modeling_outputs import BaseModelOutput
from transformers import T5Model, T5Tokenizer, T5Config, T5ForConditionalGeneration, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import corpus_bleu
from IPython.display import clear_output

import wandb

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
with open('data.txt', 'r') as f:
    texts = f.read()

texts = texts.split(sep='\n')
texts = [row.split('\t') for row in texts]
texts_en = [row[0] for row in texts if len(row) == 2]
texts_ru = [row[1] for row in texts if len(row) == 2]

print('Num texts:', len(texts_en), len(texts_ru))
print('En max len:', max([len(row) for row in texts_en]))
print('Ru max len:', max([len(row) for row in texts_ru]))

Num texts: 50000 50000
En max len: 518
Ru max len: 431


In [6]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LEN = 518
DEVICE

device(type='cuda')

In [7]:
class TextDataset(Dataset):
    def __init__(self, texts_en, texts_ru):
        self.texts_en = texts_en
        self.texts_ru = texts_ru
        
    def __len__(self):
        return len(self.texts_en)
    
    def __getitem__(self, idx):
        return self.texts_en[idx], self.texts_ru[idx]

In [8]:
train_texts_en, val_texts_en, train_texts_ru, val_texts_ru = train_test_split(texts_en, texts_ru, test_size=0.05, random_state=42)
train_texts_en, test_texts_en, train_texts_ru, test_texts_ru = train_test_split(train_texts_en, train_texts_ru, test_size=0.05, random_state=42)

train_dataset = TextDataset(train_texts_en, train_texts_ru)
val_dataset = TextDataset(val_texts_en, val_texts_ru)
test_dataset = TextDataset(test_texts_en, test_texts_ru)

In [9]:
n_epochs = 4
batch_size = 4
log_each_n_iterations = 200
generate_n = 1

In [10]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

train_loader = DataLoader(train_dataset, batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size)
test_loader = DataLoader(test_dataset, batch_size)
generate_loader = DataLoader(val_dataset, generate_n, shuffle=True)


model_name = 'Helsinki-NLP/opus-mt-en-ru'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(DEVICE)

LR = 1e-6
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [11]:
def decode(x, max_length=MAX_LEN, repetition_penalty=3.0, **kwargs):
    with torch.no_grad():
        out = model.generate(
            **x, 
            max_length=max_length, 
            repetition_penalty=repetition_penalty,
            **kwargs
        )
        return [tokenizer.decode(tokens, skip_special_tokens=True).replace('▁', ' ') for tokens in out]
    
    

def calc_bleu(loader):
    original_text = []
    generated_text = []
    model.eval()
    for en, ru in tqdm(loader):
        x = tokenizer(en, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)
        with torch.no_grad():
            out = model.generate(**x, repetition_penalty=3.0)
            generated = [tokenizer.decode(tokens, skip_special_tokens=True) for tokens in out]
            original_text.extend(ru)
            generated_text.extend(generated)
    return corpus_bleu([[text] for text in original_text], generated_text) * 100

In [12]:
calc_bleu(test_loader)

  0%|          | 0/594 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 594/594 [03:56<00:00,  2.51it/s]


46.7290569363296

In [13]:
wandb.init(
    # set the wandb project where this run will be logged
    project="nlp-lab2",
    entity='naumenko-km',
    notes="helsinki",
    name='helsinki_09',
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LR,
    "encoder": model_name,
    "decoder": model_name,
    "epochs": n_epochs,
    }
)

best_bleu = 0
val_bleu = 0
mean_val_loss = 10
iters = 1
bad_texts = [
    'In the village of Ystalyfera, Ty-Ni Guest House lies just outside the Brecon Beacons National Park and overlooks the Darren Mountain. Around 30 minutes’ drive from Swansea and Mumbles, this guest house provides free parking and WiFi.',
    'A microwave and a small fridge are included in each room at the Sun Bridge Inn Pine Bluff. A cable TV is also provided.'
]

for i in range(1, n_epochs + 1):
    print(f'[EPOCH {i}]')
    tqdm_iterator = tqdm(train_loader)
    for text_en_batch, text_ru_batch in tqdm_iterator:
        model.train()
        x = tokenizer(text_en_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)
        y = tokenizer(text_ru_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)

        y.input_ids[y.input_ids == 0] = -100  # не учитываем паддинг
        loss = model(
            **x,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({"batch loss": loss.item(), "val loss": mean_val_loss, 'val bleu': val_bleu}, step=iters)

        if iters % log_each_n_iterations == 0:
            model.eval()
            en, ru = next(iter(generate_loader))    
            x = tokenizer(en, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)
            out = model.generate(**x, repetition_penalty=3.0)
            generated = [tokenizer.decode(tokens, skip_special_tokens=True) for tokens in out]
            
            example = wandb.Html(data=f'batches: {iters} <br> True: {ru[0]} <br> Generated: {generated[0]}')
            wandb.log({"texts": example}, step=iters)
        iters += 1

    tqdm_iterator = tqdm(val_loader)
    val_loss = []
    for text_en_batch, text_ru_batch in tqdm_iterator:
        model.eval()
        with torch.no_grad():
            x = tokenizer(text_en_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)
            y = tokenizer(text_ru_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)

            y.input_ids[y.input_ids == 0] = -100  # не учитываем паддинг
            loss = model(
                **x,
                labels=y.input_ids,
                decoder_attention_mask=y.attention_mask,
                return_dict=True
            ).loss
            val_loss.append(loss.item())
    
    mean_val_loss = np.mean(val_loss)
    val_bleu = calc_bleu(val_loader)
    print("val loss:", mean_val_loss, 'val bleu:', val_bleu)
    scheduler.step()
    
    if val_bleu > best_bleu:
        best_bleu = val_bleu
        # torch.save(model.state_dict(), 'model_train.pt')
        print(f'best loss improved!')

wandb.finish()

wandb: Currently logged in as: naumenko-km. Use `wandb login --relogin` to force relogin


[EPOCH 1]


100%|██████████| 625/625 [43:03<00:00,  4.13s/it]


val loss: 0.6486102778911591 val bleu: 42.02435744372206
best loss improved!
[EPOCH 2]


100%|██████████| 625/625 [51:17<00:00,  4.92s/it]


...

100%|██████████| 625/625 [51:52<00:00,  4.98s/it]


val loss: 0.511704444694519 val bleu: 23.740646338113578
[EPOCH 4]


100%|██████████| 625/625 [52:10<00:00,  5.01s/it]


val loss: 0.49706644332408906 val bleu: 22.64839369753393


batch loss,█▇▅▆▅▄▃▃▂▃▃▃▃▃▂▂▃▃▂▃▂▃▂▃▃▃▁▄▂▃▁▃▁▂▂▂▂▁▃▃
val bleu,▁▁▁▁▁▁▁▁▁▁██████████▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅
val loss,██████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch loss,0.28102
val bleu,23.74065
val loss,0.5117


In [14]:
calc_bleu(test_loader)

100%|██████████| 594/594 [49:03<00:00,  4.96s/it]


23.259726473443617